# Scopes and Namespaces

In [ ]:
def scope_test():
    def do_local():
        spam = "local spam"

    def do_nonlocal():
        nonlocal spam
        spam = "nonlocal spam"

    def do_global():
        global spam
        spam = "global spam"

    spam = "test spam"
    do_local()
    print("After local assignment:", spam)
    do_nonlocal()
    print("After nonlocal assignment:", spam)
    do_global()
    print("After global assignment:", spam)

scope_test()
print("In global scope:", spam)

# Iterator

In [7]:
# when "for i in list" is called
# iter() is applied to the list to make it an iterator
# and next() is called until StopIteration Exception is raised
iterator = iter([10,20,30,40])

In [8]:
next(iterator)

10

In [9]:
next(iterator)

20

# Generator

## Simple Example

In [11]:
def animal_generator():
    yield "anaconda"
    yield "bat"
    yield "cat"

In [13]:
generator = animal_generator()
print(next(generator))
print(next(generator))
print(next(generator))

anaconda
bat
cat


## Infinite Generator

In [21]:
# like a function, but the state of the variable
# is stored between calls
# returns value when yield is reached
# continues after yield when called again
# *** having a return is similar to raising a StopIteration Exception
def counter():
    start = 0
    while True:
        yield start
        start += 1

In [22]:
count = counter()
for i in range(5):
    print(next(count))

0
1
2
3
4


In [23]:
# iterate to infinity if no exit condition
for i in count:
    if i == 1000:
        print(i)
        break

1000


## Sending / Receiving

In [6]:
from random import choice

In [7]:
def song_generator(song_list):
    new_song = None
    while True:
        
        # if a new song is sent to the generator
        # append it to all songs
        if new_song != None:
            if new_song not in song_list:
                song_list.append(new_song)
        
        # when next() is called
        # it will recommend a song randomly from song list
        # LHS: receiving variable
        # RHS: yielding variable
        new_song = yield choice(song_list)

In [32]:
gen = song_generator(["A", "B", "C", "D", "E"])

In [33]:
# must run yield statement for the first time
# next() sends None to the generator
next(gen)

'E'

In [34]:
gen.send("Z")

'Z'

## throw()

In [35]:
def counter(first_value=0, step=1):
    count = first_value
    
    while True:
        try:
            new_first_value = yield count
            if new_first_value == None:
                count += step
            else:
                count = new_first_value
        except Exception:
            yield ("The current count is:", count)

In [41]:
counter1 = counter()
next(counter1)

0

In [42]:
for i in range(5):
    next(counter1)
    
counter1.throw(Exception)

('The current count is:', 5)

In [43]:
# ability to 'interrupt' the iteration with an exception
# and yield some values, then continue
for i in range(5):
    next(counter1)
    
counter1.throw(Exception)

('The current count is:', 9)

## yield from

Substitute a for-loop

In [44]:
def generator():
    yield from "Python"

In [45]:
gen = generator()

for i in gen:
    print(i, end=" ")

P y t h o n 

# Decorators

### Syntax of General Decorator

A function that takes in and returns a function while adding functionalities and preserving its code.

In [8]:
def decorator(func):
    """Calls the original function with arbitrary inputs,
    with added functionality of printing a statement."""
    
    print(f"Function {func.__name__} has been decorated.")
    
    def decorated_function(*args, **kwargs):
        """Original function with added functionality.
        """
        print(f"The decorated version of {func.__name__} has been called.") # added print functionality
        return func(*args, **kwargs)
    
    return decorated_function

One way of applying the decorator:

In [2]:
def func(a, b, c, d):
    return (a + b)*(c + d)

decorated_func = decorator(func)

Function func has been decorated.


In [3]:
decorated_func(1, 2, c=3, d=4)

The decorated version of func has been called.


21

Another way:

In [9]:
@decorator
def func(a, b, c, d):
    return (a + b)*(c + d)

Function func has been decorated.


In [10]:
func(1, 2, c=3, d=4)

The decorated version of func has been called.


21

The wrapper's details replaces that of the original function.

In [12]:
print(func.__name__, func.__doc__, func.__module__, sep="\n")

decorated_function
Original function with added functionality.
        
__main__


### Multiple Decorators

With multiple decorators, the order of wrapping the function is 1,2,3 and order of calling the wrappers is 3,2,1.

In [ ]:
@deco3
@deco2
@deco1
def func(x):
    pass

### Common use cases

1) Checking / validating arguments
2) Counting function calls

### Decorators with Parameters

Add an additional 'layer' of function definition to receive parameters.

In [ ]:
def customized_wording(verb):
    
    def decorator(func):
        """Calls the original function with arbitrary inputs,
        with added functionality of printing a statement."""

        print(f"Function {func.__name__} has been {verb}.")

        def decorated_function(*args, **kwargs):
            print(f"The {verb} version of {func.__name__} has been called.") # added print functionality
            return func(*args, **kwargs)

        return decorated_function
    
    return decorator

In [ ]:
@customized_wording("beautified")
def func(a, b, c, d):
    return (a + b)*(c + d)

In [ ]:
func(1, 2, c=3, d=4)

### @wraps

Preserves .\__name__, .\__doc__, .\__module__ of wrapped function.

In [13]:
from functools import wraps

In [16]:
def decorator(func):
    @wraps(func) # add this decorator
    def decorated_function(*args, **kwargs):
        """Original function with added functionality.
        """
        print(f"The decorated version of {func.__name__} has been called.") # added print functionality
        return func(*args, **kwargs)
    
    return decorated_function

In [17]:
@decorator
def func(a, b, c, d):
    """Returns (a+b)*(c+d)"""
    return (a + b)*(c + d)

In [18]:
print(func.__name__, func.__doc__, func.__module__, sep="\n")

func
Returns (a+b)*(c+d)
__main__
